# 서포트 벡터 머신(SVM)과 특성 스케일링의 중요성

이 노트북은 대표적인 분류 알고리즘 중 하나인 **서포트 벡터 머신(Support Vector Machine, SVM)**의 특징을 알아보고, 모델 성능에 **특성 스케일링(Feature Scaling)**이 얼마나 중요한 영향을 미치는지 직접 확인하는 것을 목표로 합니다.

### 서포트 벡터 머신이란?
SVM은 각기 다른 클래스에 속한 데이터 포인트들 사이에 가장 넓은 '경계'(margin)를 찾는 방식으로 데이터를 분류하는 강력한 알고리즘입니다. 이 경계를 찾는 과정은 데이터 포인트들 간의 거리를 기반으로 하기 때문에, 각 특성(feature)의 값 범위(scale)가 다르면 특정 특성이 모델에 과도한 영향을 미칠 수 있습니다.

### 학습 과정:
1.  유방암 진단 데이터셋을 로드합니다.
2.  데이터를 훈련 세트와 테스트 세트로 분할합니다.
3.  **스케일링을 하지 않은** 원본 데이터로 로지스틱 회귀와 SVM 모델을 각각 학습하고 성능을 비교합니다.
4.  `StandardScaler`를 사용하여 데이터를 **스케일링**합니다.
5.  스케일링된 데이터로 SVM 모델을 다시 학습하고, 스케일링 전의 성능과 비교합니다.

### 1. 라이브러리 임포트

In [ ]:
# 데이터셋 로드
from sklearn.datasets import load_breast_cancer

# 데이터 전처리 및 분할
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 모델
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

### 2. 데이터 로드 및 분할

scikit-learn의 유방암 진단 데이터셋을 불러온 후, 모델 학습과 평가를 위해 훈련 세트와 테스트 세트로 분할합니다.

**중요**: 데이터 유출(Data Leakage)을 방지하기 위해, 스케일링과 같은 모든 전처리 과정은 반드시 데이터를 훈련/테스트 세트로 **분할한 후에** 진행해야 합니다. 훈련 세트에서 학습한 스케일러를 테스트 세트에 동일하게 적용합니다.

In [ ]:
# 데이터 로드
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(f"원본 데이터 형태: {X.shape}")
print(f"훈련 데이터 형태: {X_train.shape}")
print(f"테스트 데이터 형태: {X_test.shape}")

### 3. 스케일링 전 모델 성능 비교

#### 3.1. 로지스틱 회귀 (스케일링 전)

In [ ]:
model_lr = LogisticRegression(max_iter=5000) # 반복 횟수를 늘려 모델이 수렴하도록 보장
model_lr.fit(X_train, y_train)

print("--- 로지스틱 회귀 (스케일링 전) ---")
print(f"훈련 세트 정확도: {model_lr.score(X_train, y_train):.4f}")
print(f"테스트 세트 정확도: {model_lr.score(X_test, y_test):.4f}")

#### 3.2. 서포트 벡터 머신 (스케일링 전)

스케일링을 적용하지 않은 원본 데이터로 SVM을 학습했을 때의 성능을 확인합니다.

In [ ]:
model_svc = SVC()
model_svc.fit(X_train, y_train)

print("--- SVM (스케일링 전) ---")
print(f"훈련 세트 정확도: {model_svc.score(X_train, y_train):.4f}")
print(f"테스트 세트 정확도: {model_svc.score(X_test, y_test):.4f}")
# 테스트 정확도가 상대적으로 낮게 나오는 것을 볼 수 있습니다.

### 4. 특성 스케일링 및 모델 재학습

#### 4.1. StandardScaler를 이용한 스케일링

`StandardScaler`를 사용하여 모든 특성의 평균을 0, 표준편차를 1로 만듭니다.
- `fit()` 또는 `fit_transform()`은 **훈련 데이터에만** 적용하여 스케일링 규칙(평균, 표준편차)을 학습합니다.
- `transform()`은 학습된 규칙을 사용하여 **훈련 데이터와 테스트 데이터 모두에** 동일하게 적용합니다.

In [ ]:
# 스케일러 객체 생성
ss = StandardScaler()

# 훈련 데이터로 스케일러를 학습시키고, 훈련 데이터를 변환합니다.
X_train_scaled = ss.fit_transform(X_train)

# 학습된 스케일러로 테스트 데이터를 변환합니다.
X_test_scaled = ss.transform(X_test)

print("스케일링 후 훈련 데이터 샘플:")
print(X_train_scaled[:1])

#### 4.2. 서포트 벡터 머신 (스케일링 후)

스케일링된 데이터로 SVM 모델을 다시 학습하고 성능을 평가합니다.

In [ ]:
model_svc_scaled = SVC()
model_svc_scaled.fit(X_train_scaled, y_train)

print("--- SVM (스케일링 후) ---")
print(f"훈련 세트 정확도: {model_svc_scaled.score(X_train_scaled, y_train):.4f}")
print(f"테스트 세트 정확도: {model_svc_scaled.score(X_test_scaled, y_test):.4f}")

### 5. 결론

- **스케일링 전 SVM**: 테스트 정확도 약 92.3%
- **스케일링 후 SVM**: 테스트 정확도 약 97.2%

결과에서 볼 수 있듯이, 특성 스케일링을 적용한 후에 SVM 모델의 성능이 **눈에 띄게 향상**되었습니다. 이는 SVM과 같은 거리 기반 알고리즘에서는 모든 특성이 동등한 스케일로 모델에 기여하도록 만드는 것이 매우 중요함을 보여줍니다.